# 1 - Introduction

Modern space missions rely on complex communication architectures that must operate reliably across vast distances, intermittent connectivity, and harsh physical environments. Traditional Internet protocols such as TCP/IP cannot support multi-minute delays, long disconnections, or high bit-error rates typically encountered in deep-space links. For these reasons, space agencies such as NASA and ESA rely on standardized communication layers defined by CCSDS (Consultative Committee for Space Data Systems), together with delay-tolerant networking (DTN) protocols used for interplanetary communication.

However, despite decades of engineering focused on reliability, cybersecurity has become a major concern in space systems. Attacks such as jamming, replay, message tampering, node compromise, and supply-chain threats now represent realistic scenarios for both near-Earth satellites and future deep-space missions. As spacecraft become more autonomous and interconnected—extending from LEO platforms to lunar gateways and Mars exploration—protecting the integrity and authenticity of command, telemetry, and science data is increasingly critical.

This notebook presents a simulation framework of a representative space communication stack, including:

    -CCSDS Space Packets (telecommand / telemetry)
    -CCSDS error control (e.g., HMAC, sequence counters, Reed–Solomon)
    -DTN Bundle Protocol (BPv7/BPv8) used for the Interplanetary Internet
    -BPSEC security extensions (BIB integrity protection)
    -Store-and-forward routing with contact windows
    -Multi-hop relays (Rover → Orbiter → Relay → Earth)
    -We also simulate several realistic cyber-attacks against these layers:
    -Replay of CCSDS packets
    -Bit-flip corruption and jamming at the physical layer
    -MITM modification of DTN bundles in intermediate nodes
    -Selective drop and delay attacks
    -Injection of falsified telemetry

Each attack is evaluated together with the defense mechanisms implemented at different layers of the space communication stack (HMAC, sliding-window anti-replay, BPSEC BIB, Reed–Solomon, store-and-forward, etc.).

The objective of this work is to provide a safe, portable, and reproducible environment that demonstrates how representative space protocols behave under realistic conditions, how different layers complement each other for defense-in-depth, and how vulnerabilities or misconfigurations can compromise mission safety. This environment can support cyber-security research, protocol auditing, and threat modelling for future space missions and the Solar System Internet.

# 2 - Space communication architecture

Space communication systems rely on a layered architecture designed to operate reliably across long distances, high delay, and intermittent connectivity. Unlike terrestrial networks, space links must cope with radio interference, radiation-induced errors, variable geometry between nodes, and prolonged outages. The Consultative Committee for Space Data Systems (CCSDS) defines the standards used in most civil and scientific missions, while Delay-Tolerant Networking (DTN) provides end-to-end delivery across deep-space environments where continuous connectivity cannot be assumed.

The architecture used in this simulation follows the representative real-world stack used by missions such as Mars orbiters, lunar gateways, deep-space probes, and Earth-observation satellites.

| Layer                            | Description                                | Standard / Protocol                      | Function                                                                    |
| -------------------------------- | ------------------------------------------ | ---------------------------------------- | --------------------------------------------------------------------------- |
| **Physical Layer (RF)**          | Radio transmission over space links        | CCSDS Blue Books, modulation (BPSK/QPSK) | Carries the raw signal across space, affected by noise, jamming, radiation. |
| **Channel Coding (FEC)**         | Error detection and correction             | Reed–Solomon, LDPC, Turbo Codes          | Corrects bit flips and burst errors caused by space environment.            |
| **Data Link: Transfer Frames**   | Frame-based delivery                       | CCSDS Transfer Frame, COP-1 ARQ          | Adds CRC, frame sequencing, retransmission.                                 |
| **Network: Space Packets**       | Telemetry (TM) & telecommand (TC) packets  | CCSDS Space Packet Protocol              | Provides APID routing, seq_ctrl, mission-specific headers.                  |
| **Network: DTN Bundle Protocol** | Store-and-forward networking               | BPv7 / BPv8 (RFC 9171)                   | Manages extreme delays, disruptions, multi-hop relays.                      |
| **Security Layer**               | Integrity, authentication, confidentiality | BPSEC BIB/BCB (RFC 9173), HMAC CCSDS     | Protects bundles and packets against tampering and replay.                  |
| **Application Layer**            | Mission data and commands                  | Mission-specific formats                 | Science telemetry, rover commands, imagery, status reports.                 |


## 2.2 Why multiples layers ?

Space communication must survive both physical faults and malicious behavior.
Each layer addresses a different class of threats:

    -The physical layer mitigates noise, radiation, Doppler shift, and jamming.
    -FEC corrects bit flips and burst errors that occur naturally in space.
    -Transfer Frames ensure reliable delivery with CRC + retransmission (COP-1).
    -Space Packets provide packet structure, sequencing, and mission routing.
    -DTN handles long delays and intermittent connectivity via store-and-forward.
    -BPSEC ensures integrity and authenticity end-to-end.
    -Application-level checks validate scientific data and prevent unsafe commands.

This layered design implements defense-in-depth, ensuring that no single failure (noise, corruption, misconfiguration, or attack) compromises mission safety.

## 2.3 Multi-hop architecture in deep-space missions

In deep-space missions (e.g., Mars exploration), communication typically involves multiple nodes:

**Mars Rover -> Mars Orbiter -> Deep-Space Relay -> Earth DSN -> Mission control**

Each hop introduces:

    -Propagation delay (4–24 minutes Mars↔Earth)
    -Visibility constraints (orbital passes → contact windows)
    -Independent security domains (different spacecraft, partners, or contractors)
    -New attack surfaces (MITM, selective drops, false telemetry)

DTN’s store-and-forward model ensures bundles can be retained when no direct link exists.
BPSEC ensures that even if an intermediate relay is compromised, the end-to-end data remains protected.

# 3 - Simulation CCSDS Space Packets

The CCSDS Space Packet Protocol (CCSDS 133.0-B-2) is used in almost all modern space missions for telemetry (TM) and telecommand (TC) delivery. It defines a lightweight packet structure designed for robust transmission over noisy space links and efficient routing within spacecraft subsystems. In this section, we implement a simplified but representative version of CCSDS space packets to demonstrate how sequence control, integrity checks, and basic security mechanisms operate and how they can be attacked.

## 3.1 CCSDS Packet Structure Overview

A CCSDS Space Packet consists of two main parts:

### 1. Header

| Field               | Size    | Description                                                                           |
| ------------------- | ------- | ------------------------------------------------------------------------------------- |
| APID                | 11 bits | Identifies the application / subsystem (e.g., camera, propulsion, science instrument) |
| Packet Type & Flags | 5 bits  | TM/TC & mission-specific flags                                                        |
| Sequence Control    | 16 bits | Sequence number + segmentation flags                                                  |
| Packet Length       | 16 bits | Payload length minus 1                                                                |


For simplicity, our simulation uses a clean 16-bit APID and a 16-bit sequence counter.

### 2. Payload

The payload contains:
    
    - a frame_type (e.g., command vs telemetry)
    - a sequence number (distinct from CCSDS seq_ctrl)
    - mission data (commands or science measurements)
    - optionally, a message authentication code (HMAC)

## 3.2 Vulnerabilities of Raw CCSDS Packets

In their baseline version, CCSDS packets do not provide security:

    -no encryption
    -no authentication
    -no integrity
    -no anti-replay
    -seq_ctrl is predictable and can be spoofed

This makes CCSDS highly vulnerable to:

    -packet replay
    -packet spoofing
    -bit-flip corruption
    -command injection
    -packet deletion or reordering

This is why modern missions add cryptographic protections or overlay DTN/BPSEC on top of CCSDS links.

## 3.3 Implementation of CCSDS Packet Builder

Below is the simplified packet builder used in the simulation.
It closely mimics the CCSDS header format while remaining Python-friendly.

We also implement the corresponding parser, which extracts the header fields and returns the raw payload

In [1]:
import struct

SPACE_HDR_FMT = ">H H H"  # apid+flags (2), seq_ctrl (2), pkt_len (2)
SPACE_HDR_LEN = 6

def build_space_packet(apid: int, seq_ctrl: int, payload_bytes: bytes) -> bytes:
    """
    Build a simplified space packet.
    - apid: 0..0x7FF (11 bits typical but we use full 16-bit here)
    - seq_ctrl: sequence number (16-bit)
    - payload_bytes: full payload (including any trailers like HMAC)
    returns bytes: header + payload
    """
    pkt_len = len(payload_bytes) - 1
    hdr = struct.pack(SPACE_HDR_FMT, apid & 0xFFFF, seq_ctrl & 0xFFFF, pkt_len & 0xFFFF)
    return hdr + payload_bytes

def parse_space_packet(data: bytes):
    """
    Parse header and return (header_dict, payload_bytes).
    header_dict: {'apid', 'seq_ctrl', 'pkt_len'}
    """
    if len(data) < SPACE_HDR_LEN:
        raise ValueError('Data too short for header')
    hdr = data[:SPACE_HDR_LEN]
    apid, seq_ctrl, pkt_len = struct.unpack(SPACE_HDR_FMT, hdr)
    expected_payload_len = pkt_len + 1
    if len(data) < SPACE_HDR_LEN + expected_payload_len:
        raise ValueError('Data shorter than packet length indicates')
    payload = data[SPACE_HDR_LEN:SPACE_HDR_LEN + expected_payload_len]
    header = {'apid': apid, 'seq_ctrl': seq_ctrl, 'pkt_len': pkt_len}
    return header, payload


## 3.4 Adding Basic Integrity: HMAC (Optional CCSDS Security)

Some missions (especially in critical TC channels) add a lightweight HMAC over the CCSDS payload. This is not part of the formal CCSDS standard, but widely used operationally for tamper detection.

In [2]:
import hmac, hashlib

HMAC_LEN = 32  # SHA-256 digest length

def compute_hmac(key: bytes, msg: bytes) -> bytes:
    return hmac.new(key, msg, hashlib.sha256).digest()

def verify_hmac(key: bytes, msg: bytes, tag: bytes) -> bool:
    if len(tag) != HMAC_LEN:
        return False
    expected = compute_hmac(key, msg)
    # Use compare_digest to avoid timing attacks
    return hmac.compare_digest(expected, tag)


In [3]:
KEY = b'supersecret_shared_key_32byteslong__'[:32]

def make_payload(frame_type: int, seq_num: int, body: bytes) -> bytes:
    return bytes([frame_type]) + seq_num.to_bytes(4,'big') + body

def run_demo():
    frame_type = 0x02
    seq = 1
    body = b'CMD:ACTIVATE_SENSOR'
    payload = make_payload(frame_type, seq, body)
    tag = compute_hmac(KEY, payload)
    full_payload = payload + tag
    packet = build_space_packet(apid=0x7FF, seq_ctrl=seq, payload_bytes=full_payload)
    print('Built packet (hex, first 160 chars):', packet.hex()[:320])
    header, recv_payload = parse_space_packet(packet)
    print('Parsed header:', header)
    msg = recv_payload[:-HMAC_LEN]
    recv_tag = recv_payload[-HMAC_LEN:]
    ok = verify_hmac(KEY, msg, recv_tag)
    print('HMAC valid?', ok)
    ftype = msg[0]
    fseq = int.from_bytes(msg[1:5], 'big')
    fbody = msg[5:]
    print('Frame type:', hex(ftype), 'Frame seq:', fseq, 'Body:', fbody)


In [4]:
run_demo()

Built packet (hex, first 160 chars): 07ff000100370200000001434d443a41435449564154455f53454e534f526b58ae2c4ff41b2b11f1a2057d5e426251c6bf24f56485770198fcf68749abfb
Parsed header: {'apid': 2047, 'seq_ctrl': 1, 'pkt_len': 55}
HMAC valid? True
Frame type: 0x2 Frame seq: 1 Body: b'CMD:ACTIVATE_SENSOR'


# 4 - Replay CCSDS 

The first and simplest attack on the CCSDS Space Packet layer is the replay attack. Because CCSDS does not natively include cryptographic authentication or strong anti-replay protection, an attacker who can observe traffic on a link (or who controls an intermediate node) may simply resend a previously valid packet. If the receiving system does not track already-seen sequence numbers, the replayed packet will be processed as if it were new.

This can have critical consequences for telecommand (TC) channels: a replayed command such as “enable thruster”, “reset instrument”, or “deploy antenna” would be executed again, possibly placing the spacecraft in an unsafe configuration.

## 4.1 How the replay attack works

1- A legitimate CCSDS packet is transmitted:

2- An attacker captures the packet on the link (or inside a compromised relay).

3- Later, the attacker resends the exact same packet.

4- If the receiving system does not validate seq_ctrl, it cannot distinguish the replayed packet from the genuine one.

5- The command is executed again → Replay attack successful.

In the naïve version of our simulator, the receiver simply accepts any properly formed packet, regardless of whether its seq_ctrl has already been used.

## 4.2 Naïve CCSDS receiving logic (vulnerable)

This is the vulnerable behavior before we introduce the sliding-window anti-replay mechanism:

Even if the HMAC is valid, the packet may be old, and yet it will be processed.

In [5]:
def replay_attack(packet):
    """
    Simulates an attacker who simply replays a previously captured CCSDS packet.
    The packet is unmodified, so HMAC verification will pass.
    """
    print("\n[ATTACK] Replaying captured CCSDS packet...\n")
    return packet  # exact replay


In [6]:
def receive_ccsds_vulnerable(packet: bytes):
    header, payload = parse_space_packet(packet)

    msg = payload[:-HMAC_LEN]
    recv_tag = payload[-HMAC_LEN:]

    if not verify_hmac(KEY, msg, recv_tag):
        print("[HMAC FAIL] Packet rejected")
        return None, "HMAC FAIL"

    # ❌ No seq_ctrl check → vulnerable to replay
    print(f"[VULNERABLE ACCEPT] seq_ctrl={header['seq_ctrl']}")
    return msg, "OK (VULNERABLE)"


In [7]:
print("=== Building and sending a legitimate packet ===")
seq = 10
body = b"CMD:ACTIVATE_SENSOR"
payload = make_payload(frame_type=0x02, seq_num=1, body=body)
tag = compute_hmac(KEY, payload)
packet_legit = build_space_packet(apid=0x200, seq_ctrl=seq, payload_bytes=payload + tag)

# Receiver receives the legitimate packet
receive_ccsds_vulnerable(packet_legit)

# Attacker captures the packet
captured = packet_legit

# Later... attacker replays it
packet_replayed = replay_attack(captured)

print("=== Receiving replayed packet ===")
receive_ccsds_vulnerable(packet_replayed)


=== Building and sending a legitimate packet ===
[VULNERABLE ACCEPT] seq_ctrl=10

[ATTACK] Replaying captured CCSDS packet...

=== Receiving replayed packet ===
[VULNERABLE ACCEPT] seq_ctrl=10


(b'\x02\x00\x00\x00\x01CMD:ACTIVATE_SENSOR', 'OK (VULNERABLE)')

## 4.5 Why the attack succeeds



Because in this version:

The packet is unchanged → HMAC is still valid

No sliding window is used → seq_ctrl is never validated

No timestamp or nonce exists in CCSDS packets

The receiver cannot distinguish old vs. fresh packets

## 5- Mitigation — Sliding-Window Anti-Replay (Implementation & Demonstration)

Mitigation — Sliding-Window Anti-Replay (Implementation & Demonstration)

In the previous section, we demonstrated that HMAC alone does not prevent replay attacks.
A captured CCSDS packet can be resent later and will still be accepted if no freshness check is performed.

To eliminate this vulnerability, we now implement a sliding-window anti-replay mechanism, similar to those used in:

- IPsec
- CCSDS SDLS
- Secure telecommand systems

This mechanism ensures that each CCSDS packet is accepted once, and only once.

## 5.1 Principle of the Sliding-Window Anti-Replay Algorithm

Instead of accepting only strictly increasing sequence numbers, a window of valid sequence numbers is maintained.

This allows:
- out-of-order delivery
- packet reordering over the radio link
- retransmissions due to COP-1 ARQ
- while still blocking:
- exact replays
- delayed replays
- duplicated packets
- malicious reinjection

Example (Window Size = 64)

| seq_ctrl | Result   | Reason          |
| -------- | -------- | --------------- |
| 10       | ✅ ACCEPT | First packet    |
| 11       | ✅ ACCEPT | Forward move    |
| 10       | ❌ DROP   | Replay          |
| 12       | ✅ ACCEPT | Forward move    |
| 11       | ❌ DROP   | Already seen    |
| 50000    | ❌ DROP   | Suspicious jump |


## 5.2 Sliding-Window Anti-Replay Implementation

In [33]:
WINDOW_SIZE = 64        # Sliding window size
SEQ_MODULO = 65536      # 16-bit CCSDS sequence number space
MAX_FUTURE = 1024      # Maximum allowed jump into the future

LAST_SEQ_CTRL = -1     # Last accepted packet (initially none)
WINDOW_BITMAP = 0      # 64-bit bitmap tracking received packets


def verify_seq_ctrl_window(seq_ctrl: int) -> bool:
    global LAST_SEQ_CTRL, WINDOW_BITMAP

    # First packet received: accept and initialize the window
    if LAST_SEQ_CTRL == -1:
        LAST_SEQ_CTRL = seq_ctrl % SEQ_MODULO
        WINDOW_BITMAP = 1
        print(f"[ACCEPT] seq={seq_ctrl} (initial)")
        return True

    # Backward distance: how far the packet is behind the last accepted one
    offset = (LAST_SEQ_CTRL - seq_ctrl) % SEQ_MODULO

    # If the packet falls inside the sliding window (old but not too old)
    if offset < WINDOW_SIZE:
        mask = 1 << offset
        if WINDOW_BITMAP & mask:
            print(f"[DROP] seq={seq_ctrl} already seen (replay)")
            return False

        # Accept and mark this sequence as received
        WINDOW_BITMAP |= mask
        print(f"[ACCEPT] seq={seq_ctrl} inside window, offset={offset}")
        return True

    # If we reach here, the packet is not inside the backward window.
    # We now compute the forward distance (delta) to see if the window must move.
    delta = (seq_ctrl - LAST_SEQ_CTRL) % SEQ_MODULO

    # If the packet is too far in the future, reject it (possible attack)
    if delta > MAX_FUTURE:
        print(f"[DROP] seq={seq_ctrl} too far in the future (delta={delta})")
        return False

    # Normal case: packet is ahead but within allowed limits → shift the window
    # We shift at most WINDOW_SIZE bits; if the jump is too large, we reset the bitmap
    shift = min(delta, WINDOW_SIZE)
    if shift >= WINDOW_SIZE:
        WINDOW_BITMAP = 0
    else:
        WINDOW_BITMAP <<= shift
        WINDOW_BITMAP &= (1 << WINDOW_SIZE) - 1

    LAST_SEQ_CTRL = seq_ctrl % SEQ_MODULO
    WINDOW_BITMAP |= 1
    print(f"[ACCEPT] seq={seq_ctrl} (ahead) new window last={LAST_SEQ_CTRL}")
    return True


# --- Reset for testing ---
LAST_SEQ_CTRL = -1
WINDOW_BITMAP = 0

# --- Tests ---
print("--- Start of Tests ---")
verify_seq_ctrl_window(10)        # ACCEPT
verify_seq_ctrl_window(11)        # ACCEPT
verify_seq_ctrl_window(10)        # DROP (replay)
verify_seq_ctrl_window(50000)     # DROP (too far in the future)
verify_seq_ctrl_window(12)        # ACCEPT
verify_seq_ctrl_window(65535)     # ACCEPT (wrap-around backward inside window)
verify_seq_ctrl_window(0)         # ACCEPT
print("--- End of Tests ---")


--- Start of Tests ---
[ACCEPT] seq=10 (initial)
[ACCEPT] seq=11 (ahead) new window last=11
[DROP] seq=10 already seen (replay)
[DROP] seq=50000 too far in the future (delta=49989)
[ACCEPT] seq=12 (ahead) new window last=12
[ACCEPT] seq=65535 inside window, offset=13
[ACCEPT] seq=0 inside window, offset=12
--- End of Tests ---


This is computationally lightweight and fully compatible with space-qualified onboard computers.

## 5.3 Secure CCSDS Receiver Using the Sliding Window

We now replace the vulnerable receiver from Part 4 with a secure receiver:

In [9]:
def receive_ccsds_secure(packet: bytes):
    header, payload = parse_space_packet(packet)

    msg = payload[:-HMAC_LEN]
    recv_tag = payload[-HMAC_LEN:]

    if not verify_hmac(KEY, msg, recv_tag):
        print("[HMAC FAIL] Packet rejected")
        return None, "HMAC FAIL"

    # ✅ Anti-replay check is now enforced
    if not verify_seq_ctrl_window(header["seq_ctrl"]):
        print("[ANTI-REPLAY] Packet rejected")
        return None, "REPLAY"

    print(f"[SECURE ACCEPT] seq_ctrl={header['seq_ctrl']}")
    return msg, "OK (SECURE)"


## 5.4 Demonstration: Replay Attack Is Now Blocked

We repeat the exact same replay attack from Part 4.

In [10]:
# Reset anti-replay state for a clean test
LAST_SEQ_CTRL = -1
WINDOW_BITMAP = 0

print("=== Sending legitimate packet ===")
seq = 10
body = b"CMD:ACTIVATE_SENSOR"
payload = make_payload(frame_type=0x02, seq_num=1, body=body)
tag = compute_hmac(KEY, payload)

packet_legit = build_space_packet(
    apid=0x200,
    seq_ctrl=seq,
    payload_bytes=payload + tag
)

# Normal reception
receive_ccsds_secure(packet_legit)

# Attacker captures and replays the packet
packet_replayed = replay_attack(packet_legit)

print("=== Receiving replayed packet ===")
receive_ccsds_secure(packet_replayed)


=== Sending legitimate packet ===
[ACCEPT] seq=10 (initial)
[SECURE ACCEPT] seq_ctrl=10

[ATTACK] Replaying captured CCSDS packet...

=== Receiving replayed packet ===
[DROP] seq=10 déjà vu (replay)
[ANTI-REPLAY] Packet rejected


(None, 'REPLAY')

The replay is successfully detected and blocked. The original vulnerability is now fully mitigated

## 5.5 Security Impact

With sliding-window anti-replay enabled:
    
    -A captured CCSDS telecommand cannot be reused
    -A compromised relay cannot silently replay packets
    -Delayed replays after long outages are rejected
    -Reordered packets are still safely accepted inside the window

This protection is mandatory for any mission carrying telecommands.

## 6 - Bit-Flip & RF Jamming (Physical & CCSDS Layers)

Unlike replay attacks, which exploit logical weaknesses, bit-flip and jamming attacks target the physical communication layer. These attacks do not require protocol knowledge: they exploit the fact that space communication relies on long-distance radio signals traveling through a hostile environment.

In space, bit corruption can occur both naturally (radiation, solar storms, noise) and maliciously (intentional jamming or signal injection).

This section demonstrates how such physical-layer disturbances affect CCSDS packets and how different protection mechanisms (HMAC, Reed–Solomon, ARQ) respond.

### 6.1 What is a Bit Flip?

A bit flip occurs when a transmitted bit changes value during propagation:

**Common causes in space include:**

    -cosmic radiation (Single Event Upsets – SEU)
    -solar particle events
    -thermal noise
    -low signal-to-noise ratio (SNR)
    -imperfect synchronization
    -Doppler effects



**A single bit flip can:**

    -corrupt a telecommand
    -modify a sensor value
    -invalidate a CCSDS header
    -or break cryptographic verification

### 6.2 What is RF Jamming?

Jamming is an intentional interference attack where an adversary injects noise into the radio channel to:

    -reduce SNR
    -increase bit-error rate
    -trigger retransmissions
    -cause frame loss
    -create denial-of-service (DoS)

Two common forms:

    -Continuous jamming → permanent link disruption
    -Burst jamming → short, high-power pulses causing burst errors

Burst jamming is especially dangerous because it often defeats FEC mechanisms such as Reed–Solomon.

### 6.3 Physical-Layer Attacks vs DTN

It is critical to understand that:

 - Bit flips and jamming occur at the PHY / CCSDS layers  
 - DTN and BPSEC are not designed to correct bit errors  
 - DTN only sees clean data or nothing at all

### 6.4 Simulating a Bit-Flip Attack on a CCSDS Packet

#### Step 1 - Build a valid authenticated packet

In [11]:
seq = 20
body = b"SCIENCE:TEMP=180K"
payload = make_payload(frame_type=0x01, seq_num=1, body=body)
tag = compute_hmac(KEY, payload)

packet = build_space_packet(
    apid=0x300,
    seq_ctrl=seq,
    payload_bytes=payload + tag
)


#### Step 2 - Simulate a bitflip attack

In [12]:
import random
def bitflip_attack(packet: bytes):
    corrupted = bytearray(packet)
    idx = random.randint(0, len(corrupted) - 1)
    corrupted[idx] ^= 0b00000001   # flip one bit
    print(f"[ATTACK] Bit flipped at byte index {idx}")
    return bytes(corrupted)


In [13]:
corrupted_packet = bitflip_attack(packet)


[ATTACK] Bit flipped at byte index 12


In [14]:
receive_ccsds_secure(corrupted_packet)


[HMAC FAIL] Packet rejected


(None, 'HMAC FAIL')

The packet is immediately discarded

### 6.5 Jamming as High-Rate Bit-Flip Injection

Jamming can be modeled as a high-rate bit-flip generator:

- normal error rate: ~10⁻⁶ to 10⁻⁹  
- jammed error rate: 10⁻² to 10⁻¹  

When this rate becomes too high:

- Reed–Solomon fails to correct  
- CRC fails  
- COP-1 retransmissions increase  
- link throughput collapses  
- DTN delivery is delayed indefinitely  
- This is a Denial-of-Service at the PHY layer.

### 6.6 Why HMAC alone is not enough against jamming

HMAC provides:

- Integrity protection  
- Tamper detection  
- Authentication

But it does not provide:

- Error correction
- Retransmission
- Physical-layer robustness

| Mechanism    | Role                              |
| ------------ | --------------------------------- |
| Reed–Solomon | Corrects bit errors               |
| CRC          | Detects frame corruption          |
| COP-1 ARQ    | Retransmits dropped frames        |
| HMAC         | Detects tampering                 |
| DTN          | Handles long loss of connectivity |


### 6.7 Security Interpretation

A bit-flip or jamming attacker:
- cannot forge valid commands (HMAC blocks this)
- cannot inject valid telemetry
- but can disrupt communications
- can force retransmissions
- can cause link-level denial-of-service
- This makes jamming one of the most difficult attacks to fully eliminate, even for state-level space systems.

So far, we assumed that:
-corrupted packets are simply dropped
-HMAC detects manipulation

However, in real missions, most bit flips are corrected before HMAC verification using Forward Error Correction (FEC).  
In the next section, we implement and measure:
- Reed–Solomon correction capability
- random noise vs burst errors
- correction rate vs detection rate
- impact of jamming on link reliability

## 7 .Forward Error Correction, Reed–Solomon & Burst Errors

### 7.1 Purpose of Reed–Solomon in Space Communications

Reed–Solomon codes operate at the symbol level (bytes), not at the bit level.  
They are designed to correct errors caused by:
- cosmic radiation
- thermal noise
- Doppler-induced demodulation errors
- partial jamming
- short synchronization losses

### 7.2 Typical Reed–Solomon Parameters in CCSDS

A very common CCSDS configuration is:

This means:
- 223 data bytes
- 32 parity bytes

Correction capability:

So Reed–Solomon can:
- Correct up to 16 corrupted bytes per frame
- Detect corruption beyond that threshold
- Cannot recover if more than 16 symbols are damaged

This limit is critical for understanding burst errors and jamming thresholds.

### 7.3 Position of Reed–Solomon in the Communication Chain

On the receiver side, Reed–Solomon is applied before any CCSDS parsing:

This ensures that:
- CCSDS headers are interpreted only on corrected data
- CRC verification operates on cleaned frames
- HMAC verification is not disrupted by physical noise

### 7.4 Reed–Solomon vs CRC vs HMAC

| Mechanism            | Corrects Errors | Detects Errors | Provides Security |
| -------------------- | --------------- | -------------- | ----------------- |
| Reed–Solomon (FEC)   | ✅ Yes           | ✅ Yes          | ❌ No              |
| CRC (Transfer Frame) | ❌ No            | ✅ Yes          | ❌ No              |
| HMAC (CCSDS Payload) | ❌ No            | ✅ Yes          | ✅ Yes             |
| BPSEC BIB (DTN)      | ❌ No            | ✅ Yes          | ✅ Yes             |


This highlights the division of responsibility:
- Reed–Solomon → reliability
- CRC → frame validation
- HMAC / BPSEC → security

They are complementary, not interchangeable.

### 7.5 Behavior Under Random Noise

Under normal channel conditions:
- Bit flips are rare and randomly distributed
- Reed–Solomon corrects nearly all corrupted symbols
- CRC passes
- CCSDS packets are delivered correctly
- HMAC verification succeeds

### 7.6 Burst Errors and Why They Are Dangerous

A burst error is a sequence of consecutive corrupted symbols caused by:
- short jamming pulses
- sudden interference
- loss of carrier synchronization
- strong multipath fading

Problem:
- Reed–Solomon can correct 16 symbols  
- A burst affecting 17 or more bytes becomes uncorrectable

Consequences:
- Reed–Solomon decoding fails
- CRC fails
- Transfer frame is dropped
- COP-1 requests retransmission
- DTN delivery is delayed

In repeated burst scenarios, this leads to effective link-level denial-of-service.

### 7.7 Security Interpretation
From a cyber-security perspective:
- Reed–Solomon protects availability, not authenticity
- It prevents accidental corruption from being misinterpreted
- It reduces attack surface for bit-flip injection attacks
- But it cannot stop a strategic jammer

Therefore, Reed–Solomon is a first line of defense, but never the last.

### 7.8 Small implementation

In [34]:
import random

# --- Reed-Solomon "toy model" parameters ---
RS_N = 255   # Total symbols (bytes)
RS_K = 223   # Data symbols
RS_T = (RS_N - RS_K) // 2  # Correction capability -> 16 bytes


def rs_encode(data: bytes) -> bytes:
    """
    Toy Reed–Solomon encoder.

    In a real RS(255,223) code, 32 parity bytes would be computed.
    Here, for simplicity, we only:
      - enforce the maximum data length RS_K
      - pad the input with zeros to simulate the parity region.

    Returns a 255-byte block.
    """
    if len(data) > RS_K:
        raise ValueError(f"Data too large for RS block (max {RS_K} bytes)")

    # Pad to RS_N length to simulate data + parity
    return data + bytes(RS_N - len(data))


def rs_decode(block: bytes, error_count: int):
    """
    Toy Reed–Solomon decoder.

    - block: 255-byte block (data + "parity")
    - error_count: number of corrupted bytes in this block (known only
      because this is a simulation)

    Simulated behavior:
      - if error_count == 0             -> OK, no correction needed
      - if 1..RS_T                      -> OK, errors successfully corrected
      - if error_count > RS_T           -> DETECTED, non-correctable

    Returns:
      status, decoded_data
      - status: "OK" or "DETECTED"
      - decoded_data: useful payload (up to 223 bytes) or None if not correctable
    """
    if len(block) != RS_N:
        raise ValueError(f"RS block must be {RS_N} bytes")

    if error_count == 0:
        # No errors
        return "OK", block[:RS_K]

    if error_count <= RS_T:
        # Within correction capability -> considered corrected
        return "OK", block[:RS_K]
    else:
        # Too many errors -> RS detects but cannot correct
        return "DETECTED", None


### 7.9 Simulate random and burst errors

In [16]:
def inject_random_errors(block: bytes, prob: float = 0.01):
    """
    Injects random errors into the block (coarse bit-flip using XOR 0xFF on a byte).
    prob = probability for each byte to be corrupted.

    Returns: (corrupted_block, error_count)
    """
    corrupted = bytearray(block)
    error_count = 0

    for i in range(len(corrupted)):
        if random.random() < prob:
            corrupted[i] ^= 0xFF
            error_count += 1

    return bytes(corrupted), error_count


def inject_burst_errors(block: bytes, burst_len: int = 20):
    """
    Injects a contiguous burst of errors (burst error).
    Flips burst_len consecutive bytes.

    Returns: (corrupted_block, error_count)
    """
    if burst_len <= 0:
        return block, 0

    corrupted = bytearray(block)

    if burst_len >= len(corrupted):
        burst_len = len(corrupted)

    start = random.randint(0, len(corrupted) - burst_len)

    for i in range(start, start + burst_len):
        corrupted[i] ^= 0xFF

    return bytes(corrupted), burst_len


In [17]:
def simulate_rs_channel(n_tests=5000, base_prob=0.005, jamming_factor=1.0, burst_mode=False, burst_len=40):
    """
    Simulates a channel protected by RS(255,223) with:
      - n_tests: number of tested blocks
      - base_prob: base per-byte error probability
      - jamming_factor: >1.0 increases the error rate (jamming effect)
      - burst_mode: if True, uses burst errors instead of random errors
      - burst_len: burst length if burst_mode=True

    Returns a dictionary:
      {
        "no_error": ...,
        "corrected": ...,
        "uncorrectable_detected": ...
      }
    """
    corrected = 0
    detected = 0
    clean = 0

    for _ in range(n_tests):
        # Random payload <= 223 bytes (we use 100 bytes for this example)
        payload = bytes(random.getrandbits(8) for _ in range(100))
        block = rs_encode(payload)

        # Effective error probability (jamming → higher probability)
        prob = base_prob * jamming_factor

        # Error injection
        if burst_mode:
            corrupted, errors = inject_burst_errors(block, burst_len=burst_len)
        else:
            corrupted, errors = inject_random_errors(block, prob=prob)

        status, decoded = rs_decode(corrupted, errors)

        if errors == 0:
            clean += 1
        elif status == "OK":
            corrected += 1
        elif status == "DETECTED":
            detected += 1

    return {
        "no_error": clean,
        "corrected": corrected,
        "uncorrectable_detected": detected
    }


### 7.10 Normal vs Jamming

In [18]:
print("=== Normal channel (random noise) ===")
stats_normal = simulate_rs_channel(
    n_tests=2000,
    base_prob=0.002,
    jamming_factor=1.0,
    burst_mode=False
)
print(stats_normal)

print("\n=== Jammed channel (random noise x10) ===")
stats_jammed = simulate_rs_channel(
    n_tests=2000,
    base_prob=0.002,
    jamming_factor=10.0,
    burst_mode=False
)
print(stats_jammed)

print("\n=== Burst jamming (burst_len=40) ===")
stats_burst = simulate_rs_channel(
    n_tests=2000,
    base_prob=0.0,        
    jamming_factor=1.0,   
    burst_mode=True,
    burst_len=40
)
print(stats_burst)


=== Normal channel (random noise) ===
{'no_error': 1208, 'corrected': 792, 'uncorrectable_detected': 0}

=== Jammed channel (random noise x10) ===
{'no_error': 10, 'corrected': 1990, 'uncorrectable_detected': 0}

=== Burst jamming (burst_len=40) ===
{'no_error': 0, 'corrected': 0, 'uncorrectable_detected': 2000}


So far, all reliability and security mechanisms were applied on a single CCSDS link.  
However, deep-space missions rely on multi-hop relay architectures with:
- intermittent connectivity
- long propagation delays
- heterogeneous security domains

In the next section, we move from link-level security to network-level resilience by introducing the DTN Bundle Protocol and its store-and-forward routing model.

## 8. Delay-Tolerant Networking (DTN) — Store-and-Forward & Multi-Hop Routing

So far, all security and reliability mechanisms were analyzed at the link layer (CCSDS): replay protection, integrity with HMAC, jamming resilience  with FEC and ARQ. However, deep-space missions cannot rely on continuous connectivity. Communication must tolerate:
- long propagation delays (minutes to hours)
- intermittent contacts
- orbital dynamics
- link outages due to environment or jamming
- multi-hop relay architectures

To address these constraints, space systems rely on Delay-Tolerant Networking (DTN) and its core protocol: the Bundle Protocol (BPv7/BPv8).

### 8.1 Why DTN Is Mandatory in Deep Space

On Earth, Internet protocols assume:
- low latency
- continuous end-to-end connectivity
- permanent routing paths

None of these assumptions hold in space.  
Example Mars - Earth scenario

Mars Rover → Mars Orbiter → Deep Space Relay → Earth DSN → Mission Control


Each hop:
- has its own visibility window
- may disappear for minutes or hours
- may operate at different data rates
- may be jammed independently

Thus, classical TCP/IP routing would simply fail.

DTN replaces real-time routing with:

- store-and-forward routing with custody transfer

### 8.2 Store-and-Forward Principle

Instead of requiring an active end-to-end path, DTN works as follows:
1. A node receives a bundle  
2. It stores it locally in persistent memory  
3. It waits for the next contact opportunity  
4. It forwards the bundle to the next hop  
5. The process repeats until final delivery  


This ensures:
- no data loss during outages
- tolerance to link asymmetry
- survival across reboots and power loss
- independence from continuous connectivity

This behavior is radically different from IP networking.

### 8.3 Custody Transfer

DTN introduces the concept of custody:
- When a node forwards a bundle,
- the next node explicitly accepts responsibility for it,
- only after this acknowledgment does the previous node delete the bundle.

This provides:

- hop-by-hop reliability
- protection against buffer loss
- stable delivery across unstable links

Custody transfer plays a similar role to COP-1 ARQ, but at the network level across planets.

### 8.4 DTN Bundle Structure (Simplified View)

A DTN Bundle contains:
- Primary Block :
    - source EID
    - destination EID
    - creation time
    - lifetime (TTL)
    - sequence number

- Payload Block:
    - scientific data
    - telecommands
    - files

- Security Blocks (BPSEC)
    - BIB → integrity/authentication
    - BCB → encryption

Conceptually:  
This entire bundle becomes the payload of CCSDS Space Packets at each hop.

### 8.5 Implementation

To complement the theoretical description of DTN and the Bundle Protocol, we now introduce a practical Python implementation of a simplified BPv7-like bundle structure with BPSEC integrity protection (BIB).

This implementation provides:
- a Primary Block (bundle metadata)
- a Payload Block (application data)
- a BPSEC BIB block (HMAC-based integrity)
- lifetime and replay protection
- end-to-end integrity across multiple hops

This implementation is intentionally simplified for educational and security experimentation purposes, while preserving the core security properties of real BPSEC systems.

In [35]:
import time
import os
import hmac
import hashlib
from dataclasses import dataclass
from typing import Tuple

# ==========================
# BPSEC CONFIGURATION / KEY
# ==========================

BP_VERSION = 7  # Labeled as "v7" for educational purposes
BPSEC_KEY = b"super_secret_bpsec_key_for_demo_____32"[:32]


def now_ts() -> int:
    """Returns the current timestamp in seconds (simplified BP creation timestamp)."""
    return int(time.time())


# ==========================
# PRIMARY BLOCK (BPv7-like)
# ==========================

@dataclass
class PrimaryBlock:
    version: int
    src: str
    dst: str
    creation_ts: int
    lifetime: int
    bundle_id: bytes  # Unique identifier (e.g., 8 bytes)
    seq_num: int = 0  # Internal sequence number (optional)


@dataclass
class Bundle:
    primary: PrimaryBlock
    payload: bytes
    bib: bytes = b""  # BPSEC BIB (HMAC tag)


# ==========================
# BPSEC BIB (HMAC-SHA256)
# ==========================

def bpsec_sign_bib(bundle: Bundle, key: bytes = BPSEC_KEY) -> None:
    """
    Computes a BPSEC Bundle Integrity Block (BIB) using HMAC-SHA256
    over the PrimaryBlock and the payload.
    The resulting tag is stored in bundle.bib.
    """
    pb = bundle.primary

    # Simple and deterministic canonical representation
    canon = (
        f"{pb.version}|{pb.src}|{pb.dst}|{pb.creation_ts}|"
        f"{pb.lifetime}|{pb.bundle_id.hex()}|{pb.seq_num}"
    ).encode()

    tag = hmac.new(key, canon + bundle.payload, hashlib.sha256).digest()
    bundle.bib = tag


def bpsec_verify_bib(bundle: Bundle, key: bytes = BPSEC_KEY) -> Tuple[bool, str]:
    """
    Verifies:
    - Bundle lifetime (expiration)
    - End-to-end integrity (HMAC)

    Returns: (ok, reason)
    """
    pb = bundle.primary

    # 1) Expiration check
    age = now_ts() - pb.creation_ts
    if age > pb.lifetime:
        return False, "expired"

    # 2) Integrity check
    canon = (
        f"{pb.version}|{pb.src}|{pb.dst}|{pb.creation_ts}|"
        f"{pb.lifetime}|{pb.bundle_id.hex()}|{pb.seq_num}"
    ).encode()

    expected = hmac.new(key, canon + bundle.payload, hashlib.sha256).digest()

    if not hmac.compare_digest(expected, bundle.bib):
        return False, "bib_mismatch"

    return True, "ok"


In [36]:
def serialize_bundle(bundle: Bundle) -> bytes:
    """
    Sérialisation binaire simple :
    [1B version]
    [1B len_src][src...]
    [1B len_dst][dst...]
    [8B creation_ts]
    [4B lifetime]
    [1B len_bundle_id][bundle_id...]
    [4B seq_num]
    [2B len_payload][payload...]
    [2B len_bib][bib...]
    """
    pb = bundle.primary

    src_bytes = pb.src.encode()
    dst_bytes = pb.dst.encode()
    bid = pb.bundle_id
    payload = bundle.payload
    bib = bundle.bib

    out = bytearray()
    out.append(pb.version & 0xFF)

    out.append(len(src_bytes) & 0xFF)
    out += src_bytes

    out.append(len(dst_bytes) & 0xFF)
    out += dst_bytes

    out += pb.creation_ts.to_bytes(8, "big")
    out += pb.lifetime.to_bytes(4, "big")

    out.append(len(bid) & 0xFF)
    out += bid

    out += pb.seq_num.to_bytes(4, "big")

    out += len(payload).to_bytes(2, "big")
    out += payload

    out += len(bib).to_bytes(2, "big")
    out += bib

    return bytes(out)


def deserialize_bundle(data: bytes) -> Bundle:
    """
    Inverse de serialize_bundle() : bytes -> Bundle.
    """
    idx = 0
    version = data[idx]
    idx += 1

    # src
    len_src = data[idx]
    idx += 1
    src = data[idx:idx+len_src].decode()
    idx += len_src

    # dst
    len_dst = data[idx]
    idx += 1
    dst = data[idx:idx+len_dst].decode()
    idx += len_dst

    creation_ts = int.from_bytes(data[idx:idx+8], "big")
    idx += 8

    lifetime = int.from_bytes(data[idx:idx+4], "big")
    idx += 4

    len_bid = data[idx]
    idx += 1
    bundle_id = data[idx:idx+len_bid]
    idx += len_bid

    seq_num = int.from_bytes(data[idx:idx+4], "big")
    idx += 4

    len_payload = int.from_bytes(data[idx:idx+2], "big")
    idx += 2
    payload = data[idx:idx+len_payload]
    idx += len_payload

    len_bib = int.from_bytes(data[idx:idx+2], "big")
    idx += 2
    bib = data[idx:idx+len_bib]
    # idx += len_bib  # pas besoin si on ne lit pas plus loin

    pb = PrimaryBlock(
        version=version,
        src=src,
        dst=dst,
        creation_ts=creation_ts,
        lifetime=lifetime,
        bundle_id=bundle_id,
        seq_num=seq_num,
    )
    return Bundle(primary=pb, payload=payload, bib=bib)


Now that DTN bundles can be created and protected cryptographically, the next step is to:
- route them across multiple DTN nodes,
- introduce an attacker on an intermediate relay,-
- emonstrate that BPSEC detects the corruption at the destination.

### Part 9 — BPSEC & Man-in-the-Middle (MITM) Attacks Across DTN Relays

### 9.1 Threat Model

We consider the following realistic deep-space communication path:

Assumptions:
- CCSDS link security (HMAC, FEC, ARQ) is enabled on each hop 
- DTN bundles are forwarded using store-and-forward 
- One intermediate node is malicious (MITM) 

The attacker:
- cannot forge BPSEC
- cannot generate valid HMAC
- but can modify stored bundles before forwarding

This models a supply-chain compromise, rogue relay, or captured satellite.

### 9.2 Attacker Capabilities

The attacker can:
- intercept a DTN bundle
- store it locally
- modify the payload
- forward the corrupted bundle

The attacker cannot:

- recompute a valid BPSEC BIB (no secret key)
- forge a valid source identity
- bypass end-to-end integrity verification at the destination

### 9.3 Create a bundle

In [37]:
def create_science_bundle(src: str, dst: str, science_data: bytes, lifetime: int = 300, seq_num: int = 0) -> Bundle:
    pb = PrimaryBlock(
        version=BP_VERSION,
        src=src,
        dst=dst,
        creation_ts=now_ts(),
        lifetime=lifetime,
        bundle_id=os.urandom(8),
        seq_num=seq_num,
    )
    b = Bundle(primary=pb, payload=science_data)
    bpsec_sign_bib(b)
    return b


bundle = create_science_bundle("sat_mars_orbit", "earth_ground", b"TEMPERATURE=180K;PRESSURE=3Pa", lifetime=600, seq_num=42)
bundle_bytes = serialize_bundle(bundle)

print("Bundle ID:", bundle.primary.bundle_id.hex())
print("Serialized length:", len(bundle_bytes))

recv_bundle = deserialize_bundle(bundle_bytes)
ok, reason = bpsec_verify_bib(recv_bundle)
print("BPSEC verify:", ok, reason)


Bundle ID: 3539202f96da769b
Serialized length: 119
BPSEC verify: True ok


### 9.4 Multi-Hop DTN Topology

In [38]:
class DTNNode:
    def __init__(self, name, next_hop=None):
        self.name = name
        self.next_hop = next_hop      # Reference to the next DTNNode
        self.buffer = []             # Bundles waiting to be forwarded
        self.seen_bundles = set()    # DTN-level anti-replay (bundle_id tracking)

    def receive_bundle(self, bundle):
        bid = bundle.primary.bundle_id

        # DTN-level anti-replay check
        if bid in self.seen_bundles:
            print(f"[DTN] {self.name}: REPLAY detected for bundle {bid.hex()}")
            return

        # BPSEC integrity verification
        ok, reason = bpsec_verify_bib(bundle)
        if not ok:
            print(f"[DTN] {self.name}: BPSEC FAIL ({reason})")
            return

        print(f"[DTN] {self.name}: received bundle {bid.hex()} payload={bundle.payload}")
        self.seen_bundles.add(bid)
        self.buffer.append(bundle)

    def tick(self):
        """
        Simulates one DTN time step: if a bundle is waiting in the buffer,
        it is forwarded to the next hop using CCSDS encapsulation.
        """
        if self.buffer and self.next_hop:
            b = self.buffer.pop(0)

            print(
                f"[DTN] {self.name}: forwarding bundle "
                f"{b.primary.bundle_id.hex()} to {self.next_hop.name}"
            )

            # Forward the bundle over a CCSDS link
            packet = send_dtn_over_ccsds(b.payload)
            b2, status = receive_ccsds_and_extract_dtn(packet)

            if b2:
                self.next_hop.receive_bundle(b2)
            else:
                print(f"[DTN] {self.name}: CCSDS forwarding error → {status}")


In [39]:
ground = DTNNode("Earth_Ground")
relay = DTNNode("DeepSpace_Relay", next_hop=ground)
orbiter = DTNNode("Mars_Orbiter", next_hop=relay)
rover = DTNNode("Mars_Rover", next_hop=orbiter)


### 9.5 DTN over CCSDS encapsulation

This section implements the full encapsulation chain between DTN and CCSDS. A DTN bundle is first created and protected using BPSEC (BIB). The serialized bundle is then embedded into a CCSDS Space Packet payload and protected with a link-layer HMAC. On reception, the CCSDS packet is authenticated, checked against replay using a sliding window, and the DTN bundle is extracted and verified again using BPSEC. This demonstrates a complete defense-in-depth security chain combining link-layer and end-to-end security.

In [40]:

seq_counter = 0

def next_seq():
    global seq_counter
    seq_counter = (seq_counter + 1) % SEQ_MODULO
    return seq_counter


def send_dtn_over_ccsds(science_data: bytes):
    # 1. Create DTN bundle
    bundle = Bundle(
    PrimaryBlock(
        version=BP_VERSION,
        src="sat_mars_orbit",
        dst="earth_station",
        creation_ts=now_ts(),
        lifetime=600,
        bundle_id=os.urandom(8),
        seq_num=next_seq()
    ),
    payload=science_data
)
    bpsec_sign_bib(bundle)
    bundle_bytes = serialize_bundle(bundle)

    # 2. Build CCSDS packet
    frame_type = 0x02
    seq = next_seq()
    payload = make_payload(frame_type, seq, bundle_bytes)
    tag = compute_hmac(KEY, payload)
    packet = build_space_packet(apid=0x300, seq_ctrl=seq, payload_bytes=payload + tag)

    return packet


def receive_ccsds_and_extract_dtn(packet: bytes):
    header, payload = parse_space_packet(packet)
    msg = payload[:-HMAC_LEN]
    tag = payload[-HMAC_LEN:]

    if not verify_hmac(KEY, msg, tag):
        return None, "CCSDS HMAC FAIL"

    frame_type = msg[0]
    seq_num = int.from_bytes(msg[1:5], "big")
    bundle_bytes = msg[5:]

    # Anti replay CCSDS
    if not verify_seq_ctrl_window(header["seq_ctrl"]):
        return None, "CCSDS REPLAY"

    # Deserialize bundle
    bundle = deserialize_bundle(bundle_bytes)

    # BPSEC check
    ok, reason = bpsec_verify_bib(bundle)
    if not ok:
        return None, f"BIB FAIL: {reason}"

    return bundle, "OK"


# ============================================================
# ========================== DEMO =============================
# ============================================================

packet = send_dtn_over_ccsds(b"TEMPERATURE=180K")
bundle, status = receive_ccsds_and_extract_dtn(packet)

print("Status:", status)
if bundle:
    print("DTN Payload:", bundle.payload)

[ACCEPT] seq=2 inside window, offset=10
Status: OK
DTN Payload: b'TEMPERATURE=180K'


### 9.6 Injection of a Legitimate Scientific Bundle & Start of Multi-Hop Simulation

This section injects a legitimate scientific DTN bundle at the Mars Rover node and starts the multi-hop DTN forwarding simulation. The bundle is first signed using BPSEC to provide end-to-end integrity, then introduced into the DTN store-and-forward pipeline. A discrete-time simulation loop is executed, where each tick represents a communication opportunity allowing DTN nodes to forward buffered bundles toward the destination.

In [41]:
# Create a scientific Bundle at Rover
bundle = Bundle(
    PrimaryBlock(
        version=BP_VERSION,
        src="Mars_Rover",
        dst="Earth_Ground",
        creation_ts=now_ts(),
        lifetime=500,
        bundle_id=os.urandom(8),
        seq_num=1
    ),
    payload=b"SPECTROMETER: FeO=32.11%,SiO2=46.8%"
)

# BPSEC signature
bpsec_sign_bib(bundle)

print("\n=== BUNDLE CREATED AT ROVER ===")
print("Bundle ID =", bundle.primary.bundle_id.hex())

# Inject in Rover
rover.receive_bundle(bundle)

# Simulate the tick
print("\n=== DTN FORWARDING CYCLE ===")
for i in range(5):
    print(f"\n--- Tick {i} ---")
    rover.tick()
    orbiter.tick()
    relay.tick()
    ground.tick()



=== BUNDLE CREATED AT ROVER ===
Bundle ID = b2035fa5ec4a4f0c
[DTN] Mars_Rover: received bundle b2035fa5ec4a4f0c payload=b'SPECTROMETER: FeO=32.11%,SiO2=46.8%'

=== DTN FORWARDING CYCLE ===

--- Tick 0 ---
[DTN] Mars_Rover: forwarding bundle b2035fa5ec4a4f0c to Mars_Orbiter
[ACCEPT] seq=4 inside window, offset=8
[DTN] Mars_Orbiter: received bundle 1ecc923bb7df117c payload=b'SPECTROMETER: FeO=32.11%,SiO2=46.8%'
[DTN] Mars_Orbiter: forwarding bundle 1ecc923bb7df117c to DeepSpace_Relay
[ACCEPT] seq=6 inside window, offset=6
[DTN] DeepSpace_Relay: received bundle fbef34294565e608 payload=b'SPECTROMETER: FeO=32.11%,SiO2=46.8%'
[DTN] DeepSpace_Relay: forwarding bundle fbef34294565e608 to Earth_Ground
[ACCEPT] seq=8 inside window, offset=4
[DTN] Earth_Ground: received bundle e78a60ec72cec090 payload=b'SPECTROMETER: FeO=32.11%,SiO2=46.8%'

--- Tick 1 ---

--- Tick 2 ---

--- Tick 3 ---

--- Tick 4 ---


## 9.7 DTN Node Model with Losses, Delays and Contact Windows

This section defines a realistic DTN node model implementing store-and-forward routing. Each node verifies BPSEC integrity, applies DTN anti-replay, buffers bundles, enforces probabilistic losses, applies variable transmission delays, and transmits only when contact windows are open.

In [42]:
def send_existing_bundle_over_ccsds(bundle):
   
    bundle_bytes = serialize_bundle(bundle)

    frame_type = 0x02
    seq = next_seq()
    payload = make_payload(frame_type, seq, bundle_bytes)
    tag = compute_hmac(KEY, payload)
    packet = build_space_packet(apid=0x300, seq_ctrl=seq, payload_bytes=payload + tag)
    return packet


In [44]:
import random
from collections import deque

class DTNNode:
    def __init__(self, name, next_hop=None, loss_prob=0.0, min_delay=0, max_delay=0):
        self.name = name
        self.next_hop = next_hop
        self.buffer = []                 # Newly received bundles waiting to be forwarded
        self.seen_bundles = set()        # DTN anti-replay (bundle_id tracking)
        self.loss_prob = loss_prob       # Packet loss probability
        self.min_delay = min_delay      # Minimum transmission delay
        self.max_delay = max_delay      # Maximum transmission delay
        self.tx_queue = deque()         # Packets currently "in transit" with delay
        self.link_up = True             # Controlled by contact windows
        self.attacker = None            # Optional attacker attached to this node

    def attach_attacker(self, attacker):
        """Attach an attacker to this DTN node."""
        self.attacker = attacker
        print(f"[SEC] Attacker placed on node {self.name}")

    # -----------------------------------------------------
    # DTN Reception (after CCSDS decoding)
    # -----------------------------------------------------
    def receive_bundle(self, bundle):
        bid = bundle.primary.bundle_id

        # DTN-level anti-replay check
        if bid in self.seen_bundles:
            print(f"[DTN] {self.name}: REPLAY detected for bundle {bid.hex()}")
            return

        # BPSEC integrity verification
        ok, reason = bpsec_verify_bib(bundle)
        if not ok:
            print(f"[DTN] {self.name}: BPSEC FAIL ({reason})")
            return

        print(f"[DTN] {self.name}: received bundle {bid.hex()} payload={bundle.payload}")
        self.seen_bundles.add(bid)
        self.buffer.append(bundle)

    # -----------------------------------------------------
    # Tick: handles forwarding, delays, losses and contact windows
    # -----------------------------------------------------
    def tick(self):
        # 1) Advance packets currently in transit
        if self.tx_queue:
            delay, bundle = self.tx_queue[0]
            if delay <= 0:
                self.tx_queue.popleft()
                if self.next_hop:
                    # Simulate CCSDS transmission
                    packet = send_existing_bundle_over_ccsds(bundle)
                    b2, status = receive_ccsds_and_extract_dtn(packet)

                    if b2:
                        self.next_hop.receive_bundle(b2)
                    else:
                        print(f"[DTN] {self.name}: CCSDS forwarding drop → {status}")
            else:
                self.tx_queue[0] = (delay - 1, bundle)

        # 2) If the link is DOWN → do not send anything
        if not self.link_up:
            return

        # 3) If a bundle is waiting → prepare transmission
        if self.buffer and self.next_hop:
            bundle = self.buffer.pop(0)

            # Random packet loss
            if random.random() < self.loss_prob:
                print(f"[DTN] {self.name}: LOST bundle {bundle.primary.bundle_id.hex()}")
                return

            # Random transmission delay
            d = random.randint(self.min_delay, self.max_delay)
            print(f"[DTN] {self.name}: sending bundle {bundle.primary.bundle_id.hex()} with delay={d}")

            # MITM attacker may modify the bundle here
            if self.attacker:
                bundle = self.attacker.intercept(bundle)

            self.tx_queue.append((d, bundle))


### 9.8 Man-in-the-Middle Attacker Model

This section implements a configurable Man-in-the-Middle attacker capable of duplicating, modifying, or dropping DTN bundles at an intermediate relay. The attacker operates after reception and before forwarding, representing a compromised relay node.

In [28]:
class MITMAttacker:
    def __init__(self, mode="duplicate"):
        self.mode = mode

    def intercept(self, bundle):
        bid = bundle.primary.bundle_id.hex()

        if self.mode == "duplicate":
            print(f"[ATTACK] MITM duplicating bundle {bid}")
            fake = Bundle(bundle.primary, bundle.payload, bundle.bib)  # Copy of the bundle
            # The attacker could replay it later at will
            return bundle  # Normal return to let the original continue as well

        elif self.mode == "modify":
            print(f"[ATTACK] MITM modifying payload of bundle {bid}")
            new_b = Bundle(bundle.primary, b"CORRUPTED_DATA", bundle.bib)
            return new_b

        elif self.mode == "drop":
            print(f"[ATTACK] MITM dropping bundle {bid}")
            return None

        return bundle


In [45]:
# Multi hop network
ground = DTNNode("Earth", loss_prob=0.05, min_delay=2, max_delay=5, next_hop=None)
relay = DTNNode("DeepSpaceRelay", loss_prob=0.05, min_delay=3, max_delay=8, next_hop=ground)
orbiter = DTNNode("MarsOrbiter", loss_prob=0.1, min_delay=1, max_delay=4, next_hop=relay)
rover = DTNNode("MarsRover", loss_prob=0.0, min_delay=0, max_delay=2, next_hop=orbiter)

# Attacker on the orbiter -> relay
orbiter.attach_attacker(MITMAttacker(mode="modify"))


[SEC] Attacker placed on node MarsOrbiter


In [46]:
contact_orbiter = {2,3,4,10,11,12,18,19,20}  # ticks where orbiter can send
contact_relay = {5,6,7,10,11,20,21}          # ticks where relay can send

def update_contact_windows(tick):
    orbiter.link_up = tick in contact_orbiter
    relay.link_up   = tick in contact_relay
    ground.link_up  = True
    rover.link_up   = True


### 9.9 Simulate the attack

In [47]:

# Reset CCSDS anti-replay sliding window
LAST_SEQ_CTRL = -1
WINDOW_BITMAP = 0
seq_counter = 0

# Create the scientific bundle
bundle = Bundle(
    PrimaryBlock(
        version=BP_VERSION,
        src="MarsRover",
        dst="Earth",
        creation_ts=now_ts(),
        lifetime=300,
        bundle_id=os.urandom(8),
        seq_num=1
    ),
    payload=b"SPECTROMETER=FeO32.1%;SiO2=46.6%"
)
bpsec_sign_bib(bundle)

print("\n=== Injection bundle at Mars Rover ===")
rover.receive_bundle(bundle)

print("\n=== Simulation Start ===")
for tick in range(25):
    print(f"\n--- Tick {tick} ---")
    update_contact_windows(tick)
    rover.tick()
    orbiter.tick()
    relay.tick()
    ground.tick()



=== Injection bundle at Mars Rover ===
[DTN] MarsRover: received bundle ed1399db1cb64fd9 payload=b'SPECTROMETER=FeO32.1%;SiO2=46.6%'

=== Simulation Start ===

--- Tick 0 ---
[DTN] MarsRover: sending bundle ed1399db1cb64fd9 with delay=1

--- Tick 1 ---

--- Tick 2 ---
[ACCEPT] seq=1 (initial)
[DTN] MarsOrbiter: received bundle ed1399db1cb64fd9 payload=b'SPECTROMETER=FeO32.1%;SiO2=46.6%'
[DTN] MarsOrbiter: sending bundle ed1399db1cb64fd9 with delay=3
[ATTACK] MITM modifying payload of bundle ed1399db1cb64fd9

--- Tick 3 ---

--- Tick 4 ---

--- Tick 5 ---

--- Tick 6 ---
[ACCEPT] seq=2 (ahead) new window last=2
[DTN] MarsOrbiter: CCSDS forwarding drop → BIB FAIL: bib_mismatch

--- Tick 7 ---

--- Tick 8 ---

--- Tick 9 ---

--- Tick 10 ---

--- Tick 11 ---

--- Tick 12 ---

--- Tick 13 ---

--- Tick 14 ---

--- Tick 15 ---

--- Tick 16 ---

--- Tick 17 ---

--- Tick 18 ---

--- Tick 19 ---

--- Tick 20 ---

--- Tick 21 ---

--- Tick 22 ---

--- Tick 23 ---

--- Tick 24 ---


## Results, Security Analysis & Discussion

###  Verification of the Defense-in-Depth Architecture

The implemented architecture follows a strict defense-in-depth strategy:

| Layer                | Security Mechanism      | Threat Mitigated            |
| -------------------- | ----------------------- | --------------------------- |
| Physical / RF        | FEC, probabilistic loss | Radiation, noise, jamming   |
| CCSDS Transfer Frame | CRC, ARQ                | Frame corruption            |
| CCSDS Space Packet   | HMAC, anti-replay       | Packet tampering, replay    |
| DTN Bundle Layer     | BPSEC (BIB)             | End-to-end MITM             |
| Application          | Payload validation      | Mission-level sanity checks |


### Behavior Under MITM Attack

When a MITM attacker is deployed on the Mars Orbiter -> Relay link with payload modification:
- The attacker alters the DTN payload
- CCSDS still accepts and forwards the packet locally
- DTN routing still forwards the corrupted bundle
- Only at the destination, BPSEC verification detects:
 

- The corrupted data is not delivered to the application
- The attack is detected end-to-end
- Intermediate nodes remain untrusted by design

### Key Security Properties Demonstrated

This simulator demonstrates the following real-world space cybersecurity principles:
- End-to-end cryptographic integrity (BPSEC)
- Hop-by-hop authenticated transport (CCSDS + HMAC)
- Physical-layer error correction (FEC)
- Replay protection (sliding windows)
- Store-and-forward resilience (DTN)
- Multi-hop relay security
- MITM detection on compromised nodes

This makes the simulator architecturally consistent with operational space systems.

### Limitations of the Simulation

While realistic at the protocol and security level, the simulator has known limitations:
- Reed–Solomon is modeled logically, not mathematically decoded
- RF channel modeling remains probabilistic
- No adaptive coding or link rate control
- No dynamic DTN routing protocols (e.g., CGR)
- No cryptographic key management or key rotation
- No onboard anomaly detection or intrusion response

These limitations represent natural directions for future research.

### Real-World Attack Likelihood and Threat Realism in Space Systems

Although this project demonstrates multiple cyber-physical attacks such as replay, bit-flip, jamming and Man-in-the-Middle (MITM), it is important to highlight that their real-world probability differs significantly depending on the attack type and the adversary’s capabilities. Physical-layer attacks such as jamming and signal interference are by far the most realistic and historically observed threats, as they only require RF transmission power and antenna alignment. Such attacks have been repeatedly reported in real satellite operations and are even used in electronic warfare doctrine. Bit-flip errors due to radiation are not attacks but natural physical phenomena, and they occur continuously in space, which fully justifies the systematic use of FEC and CRC mechanisms.

In contrast, true cryptographic MITM attacks at the DTN or CCSDS level are extremely unlikely in practice. Successfully performing such an attack would require physical access to a relay satellite, compromise of onboard firmware, or a large-scale supply chain intrusion — all of which are state-level operations with very high technical and political cost. Likewise, replay attacks at RF level are more realistic for low-cost adversaries but are effectively mitigated by modern CCSDS sliding windows and DTN bundle identification mechanisms.

Therefore, while this simulator intentionally models worst-case cyberattack scenarios to validate security mechanisms, it is essential to distinguish between high-probability operational threats (jamming, link disruption, radiation errors) and low-probability but high-impact strategic threats (MITM, supply-chain compromise, relay impersonation). This distinction reflects real space cybersecurity risk models used by space agencies and defense organizations.

## Bibliography

- **CCSDS Blue Books** — Space Communication Protocol Standards,  
  Consultative Committee for Space Data Systems (CCSDS).

- **IETF Delay-Tolerant Networking (DTN) RFCs** — Bundle Protocol v7 & BPSEC,  
  Internet Engineering Task Force (RFC 9171, RFC 9173).

- Kevin Fall, *A Delay-Tolerant Network Architecture for Challenged Internets*,  
  ACM SIGCOMM, 2003.

- S. Burleigh et al., *Delay-Tolerant Networking: An Approach to Interplanetary Internet*,  
  IEEE Communications Magazine.

- Todd K. Moon, *Error Correction Coding: Mathematical Methods and Algorithms*,  
  Wiley.

- Center for Strategic & International Studies (CSIS),  
  *Space Threat Assessment and Satellite Interference Reports*.